In [154]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import tools
import itertools

from sklearn import manifold
from sklearn import preprocessing
from sklearn import mixture

In [155]:
from pandas import set_option
set_option("display.max_rows", 10)
pd.options.mode.chained_assignment = None

np.random.seed(0)

filename = '../training_data.csv'
training_data = pd.read_csv(filename)
training_data

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,3,A1 SH,SHRIMPLIN,2793.0,77.450,0.664,9.900,11.915,4.600,1,1.000
1,3,A1 SH,SHRIMPLIN,2793.5,78.260,0.661,14.200,12.565,4.100,1,0.979
2,3,A1 SH,SHRIMPLIN,2794.0,79.050,0.658,14.800,13.050,3.600,1,0.957
3,3,A1 SH,SHRIMPLIN,2794.5,86.100,0.655,13.900,13.115,3.500,1,0.936
4,3,A1 SH,SHRIMPLIN,2795.0,74.580,0.647,13.500,13.300,3.400,1,0.915
...,...,...,...,...,...,...,...,...,...,...,...
3227,5,C LM,CHURCHMAN BIBLE,3120.5,46.719,0.947,1.828,7.254,3.617,2,0.685
3228,5,C LM,CHURCHMAN BIBLE,3121.0,44.563,0.953,2.241,8.013,3.344,2,0.677
3229,5,C LM,CHURCHMAN BIBLE,3121.5,49.719,0.964,2.925,8.013,3.190,2,0.669
3230,5,C LM,CHURCHMAN BIBLE,3122.0,51.469,0.965,3.083,7.708,3.152,2,0.661


In [177]:
def build_gmm(features, n_components):
    X = prep_features(features)
    gmm = mixture.BayesianGaussianMixture(n_components=n_components,
                                          init_params="kmeans",
                                          covariance_type='full',
                                          weight_concentration_prior=10000)
    gmm.fit(X)
    return gmm

def build_gmm_per_class(features, list_of_labels, n_components):
    models = {}
    for L in unique_labels:        
        F = features.loc[L]
        N = min(len(F), n_components)
        models[L] = build_gmm(F, N)
    return models

def prep_features(data):
    X = preprocessing.RobustScaler().fit_transform(data.values)
    X = manifold.LocallyLinearEmbedding(n_components=5).fit_transform(X)
    return X

def build_models(features, N):
    return build_gmm_per_class(features, unique_labels, N)

In [178]:
features = training_data.drop(['Depth','Well Name','Formation','Facies','NM_M','RELPOS'], axis=1)
features

,GR,ILD_log10,DeltaPHI,PHIND,PE
0,77.450,0.664,9.900,11.915,4.600
1,78.260,0.661,14.200,12.565,4.100
2,79.050,0.658,14.800,13.050,3.600
3,86.100,0.655,13.900,13.115,3.500
4,74.580,0.647,13.500,13.300,3.400
...,...,...,...,...,...
3227,46.719,0.947,1.828,7.254,3.617
3228,44.563,0.953,2.241,8.013,3.344
3229,49.719,0.964,2.925,8.013,3.190
3230,51.469,0.965,3.083,7.708,3.152


In [179]:
facies = training_data['Facies']
facies

0       3
1       3
2       3
3       3
4       3
       ..
3227    5
3228    5
3229    5
3230    5
3231    5
Name: Facies, dtype: int64

In [180]:
unique_labels = training_data['Facies'].unique();

In [ ]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn import metrics

facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00',
       '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']

logo = LeaveOneGroupOut()

cmap_facies = colors.ListedColormap(facies_colors[0:len(facies_colors)], 'indexed')

print(features)

wells = training_data["Well Name"].values
for train, test in logo.split(features, facies, groups=wells):
    train_facies = facies.iloc[train]
    train_features = features.iloc[train]
    train_features['Facies'] = pd.Series(train_facies, index=train_features.index)
    train_features = train_features.set_index(['Facies'])
    models = build_models(train_features, 60)
    test_features = features.iloc[test]
    P = np.zeros((9, len(test)))
    for label, model in models.items():
        Pyy = np.array(model.predict_proba(prep_features(test_features)))
        P[label-1, :] = np.amax(Pyy, axis=1)
    yy = np.argmax(P, axis=0)+1
    well_name = wells[test[0]]
    score = metrics.f1_score(facies[test], yy, average='weighted')
    print("{:>20s}  {:.3f}".format(well_name, score))

          GR  ILD_log10  DeltaPHI   PHIND     PE
0     77.450      0.664     9.900  11.915  4.600
1     78.260      0.661    14.200  12.565  4.100
2     79.050      0.658    14.800  13.050  3.600
3     86.100      0.655    13.900  13.115  3.500
4     74.580      0.647    13.500  13.300  3.400
...      ...        ...       ...     ...    ...
3227  46.719      0.947     1.828   7.254  3.617
3228  44.563      0.953     2.241   8.013  3.344
3229  49.719      0.964     2.925   8.013  3.190
3230  51.469      0.965     3.083   7.708  3.152
3231  50.031      0.970     2.609   6.668  3.295

[3232 rows x 5 columns]


/Users/stevejpurves/anaconda3/lib/python3.5/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/Users/stevejpurves/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


     CHURCHMAN BIBLE  0.021


/Users/stevejpurves/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


      CROSS H CATTLE  0.088
            LUKE G U  0.049
               NEWBY  0.112
               NOLAN  0.095


In [ ]:
def plot_facies_pair(test, pred):
    f, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 12))
    cluster_test=np.repeat(np.expand_dims(y[test],1), 100, 1)
    cluster_pred=np.repeat(np.expand_dims(yy,1), 100, 1)
    im=ax[0].imshow(cluster_test, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=1,vmax=9)
    im=ax[1].imshow(cluster_pred, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=1,vmax=9)
    ax[0].set_xlabel('Test')
    ax[1].set_xlabel('Pred')